In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
import pyspark.sql.functions as f

# This is SparkCore and contains RDD's and their transformations and actions 
sc = SparkContext()

# This is SparkSQL and contains DataFrame and their transformations and actions 
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/11/27 17:29:58 WARN Utils: Your hostname, DESKTOP-E24BUDU resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/11/27 17:29:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/27 17:29:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/27 17:29:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [12]:
# creating an RDD from a python array 
array = [1,2.4,3,4,5.2]
my_rdd = sc.parallelize(array) # creates a collection RDD which can hold various types of items (ints and floats in this case)
print(my_rdd) 

ParallelCollectionRDD[8] at readRDDFromFile at PythonRDD.scala:289


In [13]:
# if you want to print an RRD you need to use the 'collect' actions which collects the partitions from the nodes to the driver and prints them
my_rdd.collect()

[1, 2.4, 3, 4, 5.2]

In [18]:
# map takes a function applies it to all elements of the RDD (its a transformation so in order to see the result we need to call an actions such as collect or count)
def add_one(x):
    return x+1
    
print(my_rdd.map(add_one).collect())
print(my_rdd.count())

[2, 3.4, 4, 5, 6.2]
5


In [ ]:
# In spark its common to use anonymous functions (lamdas) for simple operations
my_rdd.map(lambda x: x+ 1).collect() # transformations and actions can be chained with . 

[2, 3.4, 4, 5, 6.2]

In [38]:
key_pair = [('a',1), ('a',2), ('b',3), ('b',4)] 
pair_rdd = sc.parallelize(key_pair) # this now represents key value pairs

In [83]:
grouped = pair_rdd.groupByKey() # this transformation returns <key, result_iterable> pairs
grouped.collect()

[('b', <pyspark.resultiterable.ResultIterable at 0x7f8122f73d90>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x7f8122f73e80>)]

In [90]:
def expand_iterable_value(x):
    new_key = x[0]
    expanded_value = [r for r in x[1]]
    return (new_key, expanded_value)

# we can exapnd the iterealbe to see them wth a simple map
expanded = grouped.map(expand_iterable_value)
expanded.collect()

# grouped.map(lambda x:(x[0], [r for r in x[1]])).collect()

[('b', [3, 4]), ('a', [1, 2])]

In [ ]:
# A pair RDD can be reduced by key
# the function should define how the two values with the same key merged
# it is applied in the following manner ...reduce(reduce(reduce(k1,k2),k3),k4)....
pair_rdd.reduceByKey(lambda v1,v2 : v1+v2).collect()

[('b', 7), ('a', 3)]

In [95]:
pair_rdd.mapValues(lambda x: x+1).collect() # this shold be straight forward now

[('a', 2), ('a', 3), ('b', 4), ('b', 5)]

In [109]:
# a text file can be read using the following
file = sc.textFile("data/text.txt")

# retruns an colletion RRD of lines
file.collect()

['hello spark',
 'this is is a spark test file',
 'this file is for test purposes']

In [110]:
# word count
file.flatMap(lambda x:[(w,1) for w in x.split()]).reduceByKey(lambda v1,v2 : v1+v2).collect()

[('this', 2),
 ('is', 3),
 ('test', 2),
 ('purposes', 1),
 ('hello', 1),
 ('spark', 2),
 ('a', 1),
 ('file', 2),
 ('for', 1)]

In [115]:
# SparkSQL offers the ability to model an DataFrames with RRDs and use SQL on them
df = spark.read.csv('data/commerce_data.csv', header=True, inferSchema=True) # header tells spark to use the first like as headers and infer schema tells spark to try and infer the type of data
df.printSchema()
df.createOrReplaceTempView("original") # this is what creates the TABLE we can use for SQL commands!
df.show(10,100)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|                        Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A| WHITE HANGING HEART T-LIGHT HOLDER|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053|                WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|     CREAM CUPID HEARTS COAT HANGER|       8|12/1/2

In [ ]:
# this is one of doing SQL queries of data frames
spark.sql("SELECT Description,  Quantity FROM original").show()

+--------------------+--------+
|         Description|Quantity|
+--------------------+--------+
|WHITE HANGING HEA...|       6|
| WHITE METAL LANTERN|       6|
|CREAM CUPID HEART...|       8|
|KNITTED UNION FLA...|       6|
|RED WOOLLY HOTTIE...|       6|
|SET 7 BABUSHKA NE...|       2|
|GLASS STAR FROSTE...|       6|
|HAND WARMER UNION...|       6|
|HAND WARMER RED P...|       6|
|ASSORTED COLOUR B...|      32|
|POPPY'S PLAYHOUSE...|       6|
|POPPY'S PLAYHOUSE...|       6|
|FELTCRAFT PRINCES...|       8|
|IVORY KNITTED MUG...|       6|
|BOX OF 6 ASSORTED...|       6|
|BOX OF VINTAGE JI...|       3|
|BOX OF VINTAGE AL...|       2|
|HOME BUILDING BLO...|       3|
|LOVE BUILDING BLO...|       3|
|RECIPE BOX WITH M...|       4|
+--------------------+--------+
only showing top 20 rows



In [ ]:
# this is another way of doing SQL queries of data frames
df.select(["Description", "Quantity"]).show()

+--------------------+--------+
|         Description|Quantity|
+--------------------+--------+
|WHITE HANGING HEA...|       6|
| WHITE METAL LANTERN|       6|
|CREAM CUPID HEART...|       8|
|KNITTED UNION FLA...|       6|
|RED WOOLLY HOTTIE...|       6|
|SET 7 BABUSHKA NE...|       2|
|GLASS STAR FROSTE...|       6|
|HAND WARMER UNION...|       6|
|HAND WARMER RED P...|       6|
|ASSORTED COLOUR B...|      32|
|POPPY'S PLAYHOUSE...|       6|
|POPPY'S PLAYHOUSE...|       6|
|FELTCRAFT PRINCES...|       8|
|IVORY KNITTED MUG...|       6|
|BOX OF 6 ASSORTED...|       6|
|BOX OF VINTAGE JI...|       3|
|BOX OF VINTAGE AL...|       2|
|HOME BUILDING BLO...|       3|
|LOVE BUILDING BLO...|       3|
|RECIPE BOX WITH M...|       4|
+--------------------+--------+
only showing top 20 rows



The following are examples of last weeks tasks using both and RDD and a DataFrame (also a pure python solution to check results)

In [29]:
# CHANGE THIS TO THE CORRECT PATH FOR YOU
path_to_commerce_data = "data/commerce_data.csv" 

In [59]:
def max_value_pair(dict):
    pairs = [(key, value) for key, value in dict.items()]
    return max(pairs, key=lambda x:x[1])

with open(path_to_commerce_data,'r') as file:
    file.readline() # throw out headers
    per_country = {}
    for line in file:
        line = line.strip().split(',')
        if len(line) != 8:
            continue
        country, item, count = line[-1], line[2], int(line[3])
        per_country[country] = per_country.get(country, {})
        per_country[country][item] = per_country[country].get(item, 0) + count

result_dict = {
    key: max_value_pair(value) 
    for key,value in per_country.items()
}

sorted(result_dict.items(), key=lambda x:x[1][1], reverse=True)

[('United Kingdom', ('WORLD WAR 2 GLIDERS ASSTD DESIGNS', 48326)),
 ('Netherlands', ('RABBIT NIGHT LIGHT', 4801)),
 ('France', ('RABBIT NIGHT LIGHT', 4023)),
 ('Japan', ('RABBIT NIGHT LIGHT', 3401)),
 ('Australia', ('MINI PAINT SET VINTAGE ', 2916)),
 ('Sweden', ('MINI PAINT SET VINTAGE ', 2916)),
 ('EIRE', ('PACK OF 72 RETROSPOT CAKE CASES', 1728)),
 ('Germany', ('ROUND SNACK BOXES SET OF4 WOODLAND ', 1218)),
 ('Spain', ('CHILDRENS CUTLERY POLKADOT PINK', 729)),
 ('Switzerland', ('PLASTERS IN TIN WOODLAND ANIMALS', 639)),
 ('Norway', ('SMALL FOLDING SCISSOR(POINTED EDGE)', 576)),
 ('Canada', ('RETRO COFFEE MUGS ASSORTED', 504)),
 ('Belgium', ('PACK OF 72 RETROSPOT CAKE CASES', 480)),
 ('Finland', ('CHILDRENS CUTLERY POLKADOT PINK', 480)),
 ('Channel Islands', ('RAIN PONCHO ', 407)),
 ('Cyprus', ('HEART DECORATION PAINTED ZINC ', 384)),
 ('Singapore', ('CHRISTMAS TREE PAINTED ZINC ', 384)),
 ('Denmark', ('RED  HARMONICA IN BOX ', 288)),
 ('Austria', ('SET 12 KIDS COLOUR  CHALK STICKS',

In [ ]:
file = sc.textFile(path_to_commerce_data)
file.collect()[0:2]

data/commerce_data.csv MapPartitionsRDD[29] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
split_file = file.map(lambda x: x.split(','))
split_file.collect()[0:2]

PythonRDD[31] at RDD at PythonRDD.scala:53

In [36]:
with_id = split_file.zipWithIndex().map(lambda x:(x[1], x[0]))
with_id.collect()[0:2]

[(0,
  ['InvoiceNo',
   'StockCode',
   'Description',
   'Quantity',
   'InvoiceDate',
   'UnitPrice',
   'CustomerID',
   'Country']),
 (1,
  ['536365',
   '85123A',
   'WHITE HANGING HEART T-LIGHT HOLDER',
   '6',
   '12/1/2010 8:26',
   '2.55',
   '17850',
   'United Kingdom'])]

In [ ]:
cleaned_data = with_id.filter(lambda x: len(x[1])==8 and x[0] != 0)
data = cleaned_data.map(lambda x:x[1])
data.collect()[0:2]

[['536365',
  '85123A',
  'WHITE HANGING HEART T-LIGHT HOLDER',
  '6',
  '12/1/2010 8:26',
  '2.55',
  '17850',
  'United Kingdom'],
 ['536365',
  '71053',
  'WHITE METAL LANTERN',
  '6',
  '12/1/2010 8:26',
  '3.39',
  '17850',
  'United Kingdom']]

In [ ]:
combined_country_item_pair  = data.map(lambda x: (x[-1]+'~'+x[2], int(x[3])))
combined_country_item_pair.collect()[0:2]

[('United Kingdom~WHITE HANGING HEART T-LIGHT HOLDER', 6),
 ('United Kingdom~WHITE METAL LANTERN', 6)]

In [ ]:
sum_combined_keys = combined_country_item_pair.reduceByKey(lambda v1,v2:v1+v2)
sum_combined_keys.collect()[0:2]

[('United Kingdom~WHITE HANGING HEART T-LIGHT HOLDER', 33193),
 ('United Kingdom~WHITE METAL LANTERN', 1779)]

In [ ]:
uncombined_item_and_country = sum_combined_keys.map(lambda x: (x[0].split('~')[0], [x[0].split('~')[1], x[1]]))
uncombined_item_and_country.collect()[0:2]

[('United Kingdom', ['WHITE HANGING HEART T-LIGHT HOLDER', 33193]),
 ('United Kingdom', ['WHITE METAL LANTERN', 1779])]

In [ ]:
listOfTotalProduct_pairs = uncombined_item_and_country.groupByKey()
listOfTotalProduct_pairs.collect()[0:2] # try expanding the iterable with a map operation + list comprehension

[('France', <pyspark.resultiterable.ResultIterable at 0x7ff6a39882b0>),
 ('Australia', <pyspark.resultiterable.ResultIterable at 0x7ff6af534490>)]

In [ ]:
country_max_bought_product  = listOfTotalProduct_pairs.map(lambda x: (x[0], max(x[1], key=lambda x:x[1])))
country_max_bought_product.collect()[0:2]

[('France', ['RABBIT NIGHT LIGHT', 4023]),
 ('Australia', ['MINI PAINT SET VINTAGE ', 2916])]

In [ ]:
final_rdd = country_max_bought_product.sortBy(lambda x: x[1][1], ascending=False)
final_rdd_flat = final_rdd.map(lambda x: (x[0],x[1][0],x[1][1]))
final_rdd_flat.collect()

[('United Kingdom', 'WORLD WAR 2 GLIDERS ASSTD DESIGNS', 48326),
 ('Netherlands', 'RABBIT NIGHT LIGHT', 4801),
 ('France', 'RABBIT NIGHT LIGHT', 4023),
 ('Japan', 'RABBIT NIGHT LIGHT', 3401),
 ('Australia', 'MINI PAINT SET VINTAGE ', 2916),
 ('Sweden', 'MINI PAINT SET VINTAGE ', 2916),
 ('EIRE', 'PACK OF 72 RETROSPOT CAKE CASES', 1728),
 ('Germany', 'ROUND SNACK BOXES SET OF4 WOODLAND ', 1218),
 ('Spain', 'CHILDRENS CUTLERY POLKADOT PINK', 729),
 ('Switzerland', 'PLASTERS IN TIN WOODLAND ANIMALS', 639),
 ('Norway', 'SMALL FOLDING SCISSOR(POINTED EDGE)', 576),
 ('Canada', 'RETRO COFFEE MUGS ASSORTED', 504),
 ('Belgium', 'PACK OF 72 RETROSPOT CAKE CASES', 480),
 ('Finland', 'CHILDRENS CUTLERY POLKADOT PINK', 480),
 ('Channel Islands', 'RAIN PONCHO ', 407),
 ('Cyprus', 'HEART DECORATION PAINTED ZINC ', 384),
 ('Singapore', 'CHRISTMAS TREE PAINTED ZINC ', 384),
 ('Denmark', 'RED  HARMONICA IN BOX ', 288),
 ('Austria', 'SET 12 KIDS COLOUR  CHALK STICKS', 288),
 ('Portugal', 'POLKADOT PEN', 

In [ ]:
rdd_to_df = spark.createDataFrame(final_rdd_flat, ["country", "description", "quanitity"])
rdd_to_df.show(10,100)

+--------------+-----------------------------------+---------+
|       country|                        description|quanitity|
+--------------+-----------------------------------+---------+
|United Kingdom|  WORLD WAR 2 GLIDERS ASSTD DESIGNS|    48326|
|   Netherlands|                 RABBIT NIGHT LIGHT|     4801|
|        France|                 RABBIT NIGHT LIGHT|     4023|
|         Japan|                 RABBIT NIGHT LIGHT|     3401|
|     Australia|            MINI PAINT SET VINTAGE |     2916|
|        Sweden|            MINI PAINT SET VINTAGE |     2916|
|          EIRE|    PACK OF 72 RETROSPOT CAKE CASES|     1728|
|       Germany|ROUND SNACK BOXES SET OF4 WOODLAND |     1218|
|         Spain|    CHILDRENS CUTLERY POLKADOT PINK|      729|
|   Switzerland|   PLASTERS IN TIN WOODLAND ANIMALS|      639|
+--------------+-----------------------------------+---------+
only showing top 10 rows



In [ ]:
df = spark.read.csv('data/commerce_data.csv', header=True, inferSchema=True)
df.printSchema()
df.createOrReplaceTempView("original")
df.show(10,100)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|                        Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A| WHITE HANGING HEART T-LIGHT HOLDER|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053|                WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|     CREAM CUPID HEARTS COAT HANGER|       8|12/1/2

In [62]:
tm = df.groupBy(["Description","Country"]) \
        .agg({"Quantity":"sum"}) \
        .withColumnRenamed("sum(Quantity)", "TotalQuantity") 

tm.createOrReplaceTempView("aggrigated") # allows using 'FROM aggrigated' in spark.sql queries

tm.show(10,100)

+-----------------------------------+--------------+-------------+
|                        Description|       Country|TotalQuantity|
+-----------------------------------+--------------+-------------+
|       HOMEMADE JAM SCENTED CANDLES|United Kingdom|        10282|
|         FAMILY PHOTO FRAME CORNICE|United Kingdom|          807|
|GREEN DRAWER KNOB ACRYLIC EDWARDIAN|United Kingdom|         1919|
|ASSORTED COLOUR LIZARD SUCTION HOOK|       Germany|           48|
|             KITTENS DESIGN FLANNEL|United Kingdom|          289|
|                     FUNKY DIVA PEN|United Kingdom|         7236|
|   ANT COPPER RED BOUDICCA BRACELET|United Kingdom|           24|
|       MIRRORED WALL ART TABLE LAMP|United Kingdom|           68|
|         SKULLS AND CROSSBONES WRAP|United Kingdom|         5500|
|       POLYESTER FILLER PAD 45x30cm|United Kingdom|          103|
+-----------------------------------+--------------+-------------+
only showing top 10 rows



In [ ]:
aggrigate = spark.sql(                            # SQL window function that ranks to all entries (partitioned by country) based on their TotalQuntity
    " SELECT Description, Country, TotalQuantity, RANK() OVER (PARTITION BY Country SORT BY TotalQuantity DESC) AS rank \
      FROM aggrigated"
)
aggrigate.show(10,200)
aggrigate.createOrReplaceTempView("aggrigate")

+---------------------------------+---------+-------------+----+
|                      Description|  Country|TotalQuantity|rank|
+---------------------------------+---------+-------------+----+
|          MINI PAINT SET VINTAGE |Australia|         2916|   1|
|               RABBIT NIGHT LIGHT|Australia|         1884|   2|
|           RED  HARMONICA IN BOX |Australia|         1704|   3|
|    RED TOADSTOOL LED NIGHT LIGHT|Australia|         1344|   4|
|             DOLLY GIRL LUNCH BOX|Australia|         1024|   5|
|     HOMEMADE JAM SCENTED CANDLES|Australia|          994|   6|
|              SPACEBOY LUNCH BOX |Australia|          960|   7|
|           BLUE HARMONICA IN BOX |Australia|          720|   8|
|             MINI JIGSAW SPACEBOY|Australia|          720|   8|
|      4 TRADITIONAL SPINNING TOPS|Australia|          700|  10|
|SET OF 6 SPICE TINS PANTRY DESIGN|Australia|          600|  11|
|        ASSORTED COLOURS SILK FAN|Australia|          600|  11|
|           KEY FOB , FRO

In [64]:
result_df = spark.sql("SELECT Description, Country, TotalQuantity FROM aggrigate WHERE rank==1")
result_df.show(10,100)

+----------------------------------+---------+-------------+
|                       Description|  Country|TotalQuantity|
+----------------------------------+---------+-------------+
|           MINI PAINT SET VINTAGE |Australia|         2916|
|  SET 12 KIDS COLOUR  CHALK STICKS|  Austria|          288|
|        ICE CREAM SUNDAE LIP GLOSS|  Bahrain|           96|
|   PACK OF 72 RETROSPOT CAKE CASES|  Belgium|          480|
|  ROSES REGENCY TEACUP AND SAUCER |   Brazil|           24|
|SET/3 RED GINGHAM ROSE STORAGE BOX|   Brazil|           24|
|    PINK REGENCY TEACUP AND SAUCER|   Brazil|           24|
|      SET OF 4 PANTRY JELLY MOULDS|   Brazil|           24|
|              DOLLY GIRL LUNCH BOX|   Brazil|           24|
|         SMALL HEART FLOWERS HOOK |   Brazil|           24|
+----------------------------------+---------+-------------+
only showing top 10 rows



In [65]:
final_result = result_df.dropDuplicates(["Country"]).sort(f.desc("TotalQuantity"))
final_result.show(10,100)

+-----------------------------------+--------------+-------------+
|                        Description|       Country|TotalQuantity|
+-----------------------------------+--------------+-------------+
|  WORLD WAR 2 GLIDERS ASSTD DESIGNS|United Kingdom|        48326|
|                 RABBIT NIGHT LIGHT|   Netherlands|         4801|
|                 RABBIT NIGHT LIGHT|        France|         4023|
|                 RABBIT NIGHT LIGHT|         Japan|         3401|
|            MINI PAINT SET VINTAGE |     Australia|         2916|
|            MINI PAINT SET VINTAGE |        Sweden|         2916|
|    PACK OF 72 RETROSPOT CAKE CASES|          EIRE|         1728|
|ROUND SNACK BOXES SET OF4 WOODLAND |       Germany|         1218|
|    CHILDRENS CUTLERY POLKADOT PINK|         Spain|          729|
|   PLASTERS IN TIN WOODLAND ANIMALS|   Switzerland|          639|
+-----------------------------------+--------------+-------------+
only showing top 10 rows

